In [1]:
from digi.xbee.devices import *
import re
import time

# Instantiate an XBee device object.
device = XBeeDevice("/dev/ttyUSB0", 9600)
device.open()

# Define callback.
def my_data_received_callback(xbee_message):
    #address = xbee_message.remote_device.get_64bit_addr()
    #data = xbee_message.data.decode("utf8")
    #print("Received data from %s: %s" % (address, data))
    remote_device = xbee_message.remote_device
    data = bytes(xbee_message.data)
    is_broadcast = xbee_message.is_broadcast
    timestamp = time.ctime(xbee_message.timestamp)
    
    data2 = []
    for i in range(0,len(data)):
        if i == 2:
            data2.append(data[i])
        elif i == 4 and chr(data[i]) != 'E':
            data2.append(data[i])
        elif i == 6 and chr(data[i]) != 'E':
            data2.append(data[i])
        elif i == 9 and chr(data[i]) != 'E':
            byte12 = data[i]*256
        elif i == 10 and chr(data[i]) != 'E':
            data2.append(data[i]+byte12)
        else:
            data2.append(chr(data[i]))
    
    data = "".join(map(str, data2))
    regex = 'SP(?P<PIR>[0-9]+|E)T(?P<DHT11_T>[0-9]+|E)H(?P<DHT11_H>[0-9]+|E)HS(?P<YL_69>[0-9]+|EE)'
    match = re.search(regex, data)
    data_dict = match.groupdict()
    remote_device = remote_device.get_parameter('NI')
    data_dict.update(nodo=remote_device, timestamp=timestamp)
    
    print("Received data from %s: %s" % (bytes(remote_device), data))
    
# Add the callback.

device.add_data_received_callback(my_data_received_callback)

InvalidOperatingModeException: Could not determine operating mode